**Affiliation with AI technology and Systems.
www.ai-techsystems.**

In [0]:
import os
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from tqdm import tqdm
import cv2

if not os.path.exists("images/validation/no"):
  os.makedirs("images/validation/no")
  
if not os.path.exists("images/validation/yes"):
  os.makedirs("images/validation/yes")

path = "images/train/"
  
for root,folders,files in os.walk(path):
  for images_folder in folders:
    
    if not os.path.isdir(os.path.join(path,images_folder)):
      continue
      
      
    for image in tqdm(os.listdir(os.path.join(path,images_folder))):
      
      temp_path = os.path.join(path , images_folder , image)
      
      if not os.path.isfile(temp_path):
        continue
      
      img = load_img(temp_path)

      data = img_to_array(img)
      
      samples = expand_dims(data, 0)
      
      datagen = ImageDataGenerator( 
          rotation_range = 15 , 
          width_shift_range=0.1,
          height_shift_range = 0.1,
          brightness_range=[0.75,1.5],
          horizontal_flip=True,
          vertical_flip = True
      )
      
      it = datagen.flow(samples, batch_size=1)
      
      for i in range(2):
        batch = it.next()
        img = batch[0].astype('uint8')
#         pyplot.imshow(img)
        save_path = "images/train/" + str(images_folder) +"/"+str(image)+"_"+str(i)+".png"
        cv2.imwrite(save_path,img)

100%|██████████| 1080/1080 [00:39<00:00, 27.37it/s]


In [0]:
path = "images/train"

train_data =[]
train_keys=[]

from tqdm import tqdm
import numpy as np
import cv2

for root,folders,files in os.walk(path):
  for i in tqdm(folders):
    if i == "yes":
      key=1
    if i == "no":
      key = 0
    
    new_path = os.path.join(root,i)
    for images in os.listdir(new_path):
      image = cv2.imread(new_path+"/"+images)
      image = np.array(image)
      
      train_data.append(image)
      train_keys.append(key)
      

100%|██████████| 2/2 [00:07<00:00,  3.40s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [0]:
path = "images/validation"

val_data =[]
val_keys=[]

from tqdm import tqdm
import numpy as np
import cv2

for root,folders,files in os.walk(path):
  for i in tqdm(folders):
    if i == "yes":
      key=1
    if i == "no":
      key = 0
    
    new_path = os.path.join(root,i)
    for images in os.listdir(new_path):
      image = cv2.imread(new_path+"/"+images)
      image = np.array(image)
      
      val_data.append(image)
      val_keys.append(key)
      

100%|██████████| 2/2 [00:01<00:00,  2.34it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [0]:
from keras.utils import to_categorical

In [0]:
for i in train_data:
  if i.shape != (228,228,3):
    print(i.shape)

In [0]:
train_data = np.array(train_data)
print(train_data.shape)
train_keys = to_categorical(np.array(train_keys),num_classes=2)
print(train_keys.shape)

val_data = np.array(val_data)
print(val_data.shape)
val_keys = to_categorical(np.array(val_keys),num_classes=2)
val_keys.shape


(5376, 228, 228, 3)
(5376, 2)
(729, 228, 228, 3)


(729, 2)

In [0]:
from keras.layers import Dense,Dropout, Conv2D ,MaxPooling2D , Input , Activation , Flatten
from keras.models import Model


In [0]:
from keras.applications.vgg16 import VGG16,preprocess_input
base_model = VGG16(weights='imagenet'
                  ,include_top=False
                  ,input_shape=(228,228,3))

W0805 10:20:34.378188 140654380124032 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 10:20:34.434328 140654380124032 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 10:20:34.446808 140654380124032 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0805 10:20:34.501749 140654380124032 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



58892288/58889256 [==============================] - 3s 0us/step


W0805 10:20:38.154100 140654380124032 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0805 10:20:38.155678 140654380124032 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



In [0]:
input = Input(shape=(228,228,3,))
output = base_model(input)
output = Flatten()(output)
output = Dropout(0.2)(output)
output = Dense(2,activation="sigmoid")(output)

W0805 10:20:53.436625 140654380124032 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model = Model(input , output)
model.layers[1].trainable=False
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 228, 228, 3)       0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 50178     
Total params: 14,764,866
Trainable params: 50,178
Non-trainable params: 14,714,688
_________________________________________________________________


In [0]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["acc"])

history = model.fit(train_data , train_keys , shuffle=True,epochs = 15 ,validation_data=[val_data,val_keys], batch_size=32 , verbose=1)

Train on 5376 samples, validate on 729 samples
Epoch 1/15
5376/5376 [==============================] - 59s 11ms/step - loss: 0.2045 - acc: 0.9855 - val_loss: 0.7889 - val_acc: 0.9479
Epoch 2/15
5376/5376 [==============================] - 58s 11ms/step - loss: 0.2201 - acc: 0.9840 - val_loss: 0.8184 - val_acc: 0.9451
Epoch 3/15
5376/5376 [==============================] - 58s 11ms/step - loss: 0.1778 - acc: 0.9870 - val_loss: 0.9026 - val_acc: 0.9383
Epoch 4/15
5376/5376 [==============================] - 58s 11ms/step - loss: 0.1762 - acc: 0.9880 - val_loss: 1.0566 - val_acc: 0.9287
Epoch 5/15
5376/5376 [==============================] - 58s 11ms/step - loss: 0.1787 - acc: 0.9865 - val_loss: 0.6865 - val_acc: 0.9554
Epoch 6/15
5376/5376 [==============================] - 58s 11ms/step - loss: 0.1753 - acc: 0.9874 - val_loss: 0.8613 - val_acc: 0.9410
Epoch 7/15
5376/5376 [==============================] - 58s 11ms/step - loss: 0.1326 - acc: 0.9902 - val_loss: 0.7273 - val_acc: 0.9506
E

In [0]:
def predicted(path):
  image = [cv2.imread(path)]
  image = np.array(image)
  prediction = model.prex`dict(image)
  z = np.argmax(prediction[0])
  return z

In [0]:
predicted(path)

In [0]:

true_test=[]
for root,folders,files in os.walk("data/test"):
  for i in folders:
    for file in os.listdir(os.path.join(root,i)):
      if i=="no":
        l = 0
      else:
        l = 1
        
      true_test.append(l)
      predicted_test.append(predicted("data/test/"+i+"/"+file))

In [0]:

score = f1_score(true_test , predicted_test)